In [ ]:
import urllib.request
import pyautogui
import time
from selenium import webdriver
from PIL import Image

In [ ]:
driver = webdriver.Chrome('./driver/chromedriver.exe')
driver.get("https://patrickhlauke.github.io/recaptcha/")
driver.fullscreen_window()

In [ ]:
driver.switch_to.default_content()
driver.find_element_by_tag_name("iframe").click()
time.sleep(0.33)

In [ ]:
# Find the right iframe in case of multiple redundant tags
iframes = [i for i in driver.find_elements_by_tag_name("iframe")]
correctIFrame = None
for i in iframes:
    driver.switch_to.frame(i)
    buttons = driver.find_elements_by_tag_name("button")
    if len(driver.find_elements_by_tag_name("button")) != 0:
        print("correct iframe found")
        correctIFrame = i
    driver.switch_to.default_content()    

In [ ]:
# Find captcha box coordinates
boxX = correctIFrame.location["x"]
boxY = 45 + correctIFrame.location["y"]
driver.switch_to.frame(correctIFrame)

def find_location_of_element(element):
    x = boxX + element.size["width"] / 2 + element.location["x"]
    y = boxY + element.size["height"] / 2 + element.location["y"]
    return x, y

In [ ]:
# Find verify button coordinates
verifyX, verifyY = find_location_of_element(driver.find_element_by_id("recaptcha-verify-button"))

# Find reload button coordinates
reloadX, reloadY = find_location_of_element(driver.find_element_by_id("recaptcha-reload-button"))

In [ ]:
def clickIndex(row, column):
    tr_elements = driver.find_elements_by_tag_name("tr")
    td_elements = tr_elements[row].find_elements_by_tag_name("td")
    td_elements[column].click()

def click(x, y):
    pyautogui.moveTo(x, y, duration=0.5)
    pyautogui.click()

def submit():
    click(verifyX, verifyY)

def reload():
    click(reloadX, reloadY)

def check_table():
    table_elements = []
    rows = driver.find_elements_by_tag_name("tr")
    for i in rows:
        columns = i.find_elements_by_tag_name("td")
        for j in columns:
            table_elements.append(j)
    return table_elements
table_elements = check_table()

In [ ]:
# Get the correct captcha size and type (3x3)
def find_3x3_captcha():
    elements = check_table()
    while len(elements) != 9:
        reload()
        time.sleep(0.5)
        elements = check_table()
    return elements

In [ ]:
table_coords = None
def get_table_coords():
    return [find_location_of_element(i) for i in find_3x3_captcha()]


# ROW AND COLUMN STARTS FROM 1 INSTEAD OF 0
def choose_captcha(row, column):
    if table_coords == None:
        print("Please use assign table_coords to get_table_coords() first.")
        return
    element_index = (row - 1) * 3 + (column - 1)
    click(table_coords[element_index][0], table_coords[element_index][1])

In [ ]:
# WARNING!! THIS CODE WILL TAKE CONTROL OF YOUR MOUSE
# USE ALT F4 ON THE CHROME DRIVER TO FORCE INTERRUPT THE KERNEL
for k in range(100):
    table_coords = get_table_coords()
    for i in range(3):
        for j in range(3):
            choose_captcha(i + 1, j + 1)
    submit()
    print("submit successful, sleeping for 3 seconds.")
    time.sleep(3)

In [ ]:
# urllib.request.urlretrieve(driver.find_element(By.TAG_NAME, "img").get_attribute("src"), "captcha.jpeg")

In [ ]:

# cropped_tiles = []

# # Iterate over the grid and crop each tile
# for i in range(3):
#     for j in range(3):
#         # Calculate the coordinates for cropping
#         left = j * tile_width
#         top = i * tile_height
#         right = left + tile_width
#         bottom = top + tile_height
        
#         # Crop the tile from the original image
#         tile = original_image.crop((left, top, right, bottom))
        
#         # Add the cropped tile to the list
#         cropped_tiles.append(tile)